<a href="https://colab.research.google.com/github/sherwyn11/Image-Classification-using-Keras/blob/master/ImageClassification(Keras).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classifier using Keras ( Bird or Dog)

# Gather Data

In [0]:
!mkdir data

In [0]:
!pip install icrawler

In [0]:
from icrawler.builtin import BingImageCrawler

In [0]:
bing_crawler = BingImageCrawler(downloader_threads=4, storage={'root_dir': 'data/dog'}) 
bing_crawler.crawl(keyword='dog', filters=None, offset=0, max_num=500)

In [0]:
bing_crawler = BingImageCrawler(downloader_threads=4, storage={'root_dir': 'data/bird'}) 
bing_crawler.crawl(keyword='bird', filters=None, offset=0, max_num=500)

# CUSTOM MODEL

In [0]:
import cv2
import numpy as np
import os

In [0]:
train_x = []
train_y = []

for file in os.listdir('/content/data/bird'):
  img = cv2.imread('/content/data/bird/' + file)
  img = cv2.resize(img,(256, 256))
  img = img / 255
  train_x.append(img)
  train_y.append(0)

In [0]:
for file in os.listdir('/content/data/dog'):
  img = cv2.imread('/content/data/dog/' + file)
  img = cv2.resize(img,(256, 256))
  img = img / 255
  train_x.append(img)
  train_y.append(1)

In [0]:
len(train_x)

In [0]:
import random

In [0]:
for i in range(len(train_x)-1, 0, -1): 
      
    j = random.randint(0, i + 1)  
    
    train_x[i], train_x[j] = train_x[j], train_x[i]
    train_y[i], train_y[j] = train_y[j], train_y[i]

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout

In [0]:
model = Sequential()

model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(2, activation = 'softmax'))

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
from keras.utils import to_categorical

In [0]:
X = [train_x]
y = [to_categorical(train_y)]

In [0]:
y

In [0]:
model.fit(X, y, epochs=15, batch_size=8)

In [0]:
!mkdir test

In [0]:
bing_crawler = BingImageCrawler(downloader_threads=4, storage={'root_dir': 'test/bird'}) 
bing_crawler.crawl(keyword='crow', filters=None, offset=0, max_num=10)

In [0]:
bing_crawler = BingImageCrawler(downloader_threads=4, storage={'root_dir': 'test/dog'}) 
bing_crawler.crawl(keyword='dog', filters=None, offset=0, max_num=10)

#TEST

0 = bird 1 = dog

In [0]:
import matplotlib.pyplot as plt

In [0]:
x = '/content/test/bird/000009.jpg'

img = cv2.imread(x)
img = cv2.resize(img,(256, 256))
img = img / 255

In [0]:
plt.imshow(cv2.imread(x))

In [0]:
if(np.argmax(model.predict([[img]])) == 0):
  print('Bird')
else:
  print('Dog')

In [0]:
x = '/content/test/dog/000002.jpg'

img = cv2.imread(x)
img = cv2.resize(img,(256, 256))
img = img / 255

In [0]:
plt.imshow(cv2.imread(x))

In [0]:
if(np.argmax(model.predict([[img]])) == 0):
  print('Bird')
else:
  print('Dog')

# Saving the model for future use

In [0]:
model.save_weights("model.h5")